---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [5]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [10]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [17]:
def graph_identification():
    
#     graph_list = []
#     for graph in P1_Graphs:
#         cluster = nx.average_clustering(graph)
#         length = nx.average_shortest_path_length(graph)
#         print(cluster,length)
#     for gf in P1_Graphs:
#         print(nx.info(gf))      
    return ['PA','SW_L','SW_L','PA','SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [7]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    X_notnull = df[df.ManagementSalary.notnull()].iloc[:,2:4].values.tolist()
    y_notnull = df[df.ManagementSalary.notnull()].iloc[:,1].values.tolist()
    X_pred = df[df.ManagementSalary.isnull()].iloc[:,2:4].values.tolist()
    index = df[df.ManagementSalary.isnull()].iloc[:,2:4].index.tolist()
    X_train, X_test, y_train, y_test = train_test_split(X_notnull, y_notnull, test_size=0.2)

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)  
    X_pred = scaler.transform(X_pred)  

    logRegClassifier = LogisticRegression()
    logRegClassifier.fit(X_train, y_train)
    #print('Training Score :\t', logRegClassifier.score(X_train, y_train))
    #print('Test Score :\t\t', logRegClassifier.score(X_test, y_test))
    #fpr, tpr, _ = roc_curve(y_test, logRegClassifier.decision_function(X_test))
    #print(auc(fpr, tpr))
    y_pred = [a[1] for a in list(logRegClassifier.predict_proba(X_pred))]
   
    return pd.Series(data=y_pred,index=index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    df = future_connections
    df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['Common Neighbors'] = df.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))

    X_notnull = df[df['Future Connection'].notnull()].iloc[:,1:4].values.tolist()
    y_notnull = df[df['Future Connection'].notnull()].iloc[:,0].values.tolist()
    X_pred = df[df['Future Connection'].isnull()].iloc[:,1:4].values.tolist()
    index = df[df['Future Connection'].isnull()].iloc[:,1:4].index.tolist()
    X_train, X_test, y_train, y_test = train_test_split(X_notnull, y_notnull, test_size=0.2)

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)  
    X_pred = scaler.transform(X_pred)  

    logRegClassifier = LogisticRegression()
    logRegClassifier.fit(X_train, y_train)
    #print('Training Score :\t', logRegClassifier.score(X_train, y_train))
    #print('Test Score :\t\t', logRegClassifier.score(X_test, y_test))
    #fpr, tpr, _ = roc_curve(y_test, logRegClassifier.decision_function(X_test))
    #print(auc(fpr, tpr))
    y_pred = [a[1] for a in list(logRegClassifier.predict_proba(X_pred))]
    
    return pd.Series(data=y_pred,index=index)